<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso1/ciclo3/1_bolsa_palabras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1e7ctPi8O3bTQoLZaO9ZZjwGr2r8Z93RS" width="100%">

# Bolsas de Palabras
---

En este notebook veremos cómo extraer características a partir de textos por medio de la estrategia de representación más común conocida como bolsa de palabras (Bag-of-Words en inglés). Comenzaremos importando las librerías de ciencia de datos necesarias para manejo de datos, visualización, y manipulación de strings.

In [ ]:
!pip install unidecode

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode
from IPython.display import display
plt.style.use("ggplot")

## **1. Motivación y Definición**
---

Las representaciones basadas en bolsas de palabras (BoW - *Bag-of-Words*) hacen parte de los métodos de representación más intuitivos y comúnmente usados en procesamiento de lenguaje natural. Consiste en encontrar una distribución de todos los términos $T=\{t_1,t_2,\dots,t_m\}$ que aparecen en un conjunto de documentos $D=\{d_1, d_2, \dots, d_n\}$, es decir, es un problema de estimación de una distribución de probabilidad: $P(T=t_j | D=d_i)$, como se muestra a continuación:

<img src="https://drive.google.com/uc?export=view&id=1rGG382UMl3M2jW9EzooibJz-G9GQ3kFu" width="100%">

Normalmente, una bolsa de palabras se calcula por medio de técnicas de conteo, buscando que cada documento esté representado por la frecuencia absoluta $f(t_j, d_i)$ (la frecuencia relativa corresponde a las probabilidades) de cada término $t_j$ en el documento $d_i$, por ejemplo, el siguiente texto:

```python
"el perro y el gato no quieren al otro perro"
```

Se puede representar numéricamente de la siguiente forma:

```python
{'el': 2, 'perro': 2, 'y': 1, 'gato': 1, 'no': 1, 'quieren': 1, 'al': 1, 'otro': 1}
```

Como puede ver, las palabras `"el"` y `"perro"` que aparecen dos veces terminan representadas por el número `2`, mientras que el resto de palabras que aparecen una única vez se representan con el número `1`.

Las bolsas de palabras se suelen usar como **representaciones basadas en histogramas** y nos permiten entrenar modelos de *machine learning*, implementar sistemas de recuperación de información, entre otras.

## **2. Implementación Paso a Paso**
---

Primero, veremos cómo podemos calcular una representación de bolsa de palabras directamente en _Python_ con ayuda de `numpy`. Para este ejemplo usaremos el siguiente corpus:

In [ ]:
corpus = [
    "el cielo es azul y hermoso",
    "me encanta este cielo azul y hermoso",
    "el zorro marron rapido salta sobre el perro perezoso",
    "el desayuno de un rey tiene salchichas jamon huevos tostadas y frijoles",
    "me encantan los huevos verdes jamon salchichas y bacon",
    "el zorro marron es rapido y el perro azul es perezoso",
    "el cielo es muy azul y el cielo es muy hermoso hoy",
    "el perro es perezoso pero el zorro marron es rapido",
]

Vamos a dividir cada documento del corpus en palabras y a convertirlas en arreglos de `numpy`, de la siguiente forma:

In [ ]:
tokens = list(map(lambda doc: np.array(doc.split()), corpus))
display(tokens)

Para el cálculo de una bolsa de palabras es necesario determinar el vocabulario (palabras únicas dentro del corpus), esto lo podemos hacer con los conjuntos de _Python_ como mostramos a continuación:

In [ ]:
doc_vocab = list(map(set, tokens))
display(doc_vocab)

Calculamos las palabras únicas con la unión `|` de los vocabularios de cada documento:

In [ ]:
vocab = set()
for words in doc_vocab:
    vocab = vocab | words
vocab = list(vocab)
display(vocab)

Con este vocabulario, creamos una tabla de referencia (LUT - *Look-Up Table*) para asociar cada palabra del corpus a un índice:

In [ ]:
word2int = pd.Series(index=list(vocab), data=np.arange(len(vocab)))
display(word2int)

Con esta tabla de referencia, podemos obtener el índice al que corresponde una palabra en específico:

In [ ]:
display(word2int["huevos"])

Para calcular conteos a partir del corpus podemos usar la función de `np.unique` con el parámetro `return_counts`. Por ejemplo, veamos los conteos de palabras para un documento:

In [ ]:
words, counts = np.unique(tokens[2], return_counts=True)
display(words)
display(counts)

Con esta función podemos estimar la bolsa de palabras para todo el corpus. Primero inicializamos una matriz de tamaño $n$ (número de documentos) por $V$ (tamaño del vocabulario):

In [ ]:
bow = np.zeros((len(corpus), len(vocab)))
display(bow)

Ahora, podemos asignar los valores de la matriz con las funciones que vimos anteriormente:

In [ ]:
for i in range(len(tokens)):
    words, counts = np.unique(tokens[i], return_counts=True)
    idx = word2int[words]
    bow[i, idx] = counts
display(bow)

Como puede observar, obtenemos una matriz numérica a partir de los textos. Esto es lo que se conoce como **embedding**. Podemos generar una visualización para uno de los documentos del conjunto de datos. Primero veamos el texto:

In [ ]:
display(corpus[2])

Esto corresponde al siguiente vector de características:

In [ ]:
display(bow[2])

Veamos un diagrama de barras con los conteos:

In [ ]:
fig, ax = plt.subplots()
ax.bar(vocab, bow[2])
ax.set_xlabel("Palabras")
ax.set_ylabel("Conteo")
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
fig.show()

## **3. Implementación de sklearn**
---

Como pudimos ver, las bolsas de palabras se pueden implementar fácilmente en _Python_, no obstante, hay implementaciones más eficientes que permiten manipular corpus más grandes y guardar los elementos de forma eficiente en la memoria.

Una de las implementaciones más típicas es el `CountVectorizer` de `sklearn`. Veamos cómo podemos importarlo:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Una de las ventajas de trabajar con el `CountVectorizer` es que funciona como cualquier transformador de `sklearn`, es decir, hace uso de métodos como `fit`, `transform` y `fit_transform`. Además de esto, puede usarse junto a utilidades como `Pipeline` o `ColumnTransformer` del mismo.

Veamos cómo podemos calcular una representación de bolsa de palabras con el dataset [Language Detection de Kaggle](https://www.kaggle.com/datasets/basilb2s/language-detection):

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u3/language.csv")
display(data.head())

Se trata de un conjunto de datos que contiene distintos textos en múltiples idiomas con su correspondiente etiqueta.

Veamos cuántos documentos tenemos por idioma:

In [ ]:
counts = data.language.value_counts()
display(counts)

Vamos a preprocesar los documentos con una función de preprocesamiento similar a la que usamos en la unidad anterior:

In [ ]:
pat = re.compile(r"[^a-z ]")
spaces = re.compile(r"\s{2,}")
def preprocess(text, min_len=1, max_len=23):
    # Normalizamos el texto
    norm_text = unidecode(text).lower()

    # Extraemos tokens
    tokens = norm_text.split()

    # Filtramos palabras por longitud
    filtered_tokens = filter(
            lambda token: len(token) >= min_len and len(token) <= max_len,
            tokens
        )
    filtered_text = " ".join(filtered_tokens)
    # Eliminamos caracteres especiales
    clean_text = re.sub(pat, "", filtered_text)
    # Eliminamos espacios duplicados
    spaces_text = re.sub(spaces, " ", clean_text)
    return spaces_text.strip()

Preprocesamos el corpus:

In [ ]:
corpus_prep = data.text.apply(preprocess).to_list()
display(corpus_prep[:5])

Con este corpus, podemos entrenar un vectorizador:

In [ ]:
vect = CountVectorizer().fit(corpus_prep)

Entre los parámetros más importantes encontramos:

- `input`: tipo de entrada de texto, puede ser `"filename"` (nombre de un archivo a cargar), `file` (fichero), `content` (variables en _Python_).
- `encoding`: tipo de [codificación](https://docs.python.org/3/library/codecs.html#standard-encodings).
- `strip_accents`: establece cómo manejar los acentos, puede ser `"ascii"` o `"unicode"` (como lo hicimos en la función de preprocesamiento.
- `lowercase`: determina si el texto se debe convertir a minúsculas.
- `preprocessor`: función que permite preprocesar el texto antes de extraer los tokens.
- `tokenizer`: función que extrae los tokens a partir de cada documento.
- `stopwords`: listado de stopwords para filtrar del texto.
- `token_pattern`: expresión regular que describe cada token.
- `ngram_range`: una tupla representando secuencias de palabras (hablaremos más en detalle de esto en el notebook de N-grams).
- `analyzer`: determina si los tokens se manejarán a nivel de `"word"` (palabra) o de `"char"` (carácter).
- `max_df`: establece la proporción máxima de documentos que contengan un término, es decir, una cuota superior del número de documentos donde puede aparecer un término.
- `min_df`: establece la proporción mínima de documentos que contengan un término.
- `max_features`: número máximo de términos a considerar en la construcción del vocabulario, se consideran los más frecuentes.
- `vocabulary`: vocabulario pre-establecido.
- `binary`: establece si se usan conteos `False` o ocurrencias `True`. En el caso de ocurrencias, el valor será 1 para todos los conteos que sean mayores que 0.

Como lo puede ver, muchos de los parámetros del `CountVectorizer` simplifican algunas partes del preprocesamiento, no obstante, comúnmente la etapa de preprocesamiento se deja aparte de la extracción de características. Podemos usar el vectorizador ajustado en la celda anterior para extraer la representación:

In [ ]:
X = vect.transform(corpus_prep)
display(X)

Como se puede ver, el resultado no es directamente un arreglo de `numpy`, se obtiene una matriz dispersa de `scipy`:

In [ ]:
display(type(X))

`sklearn` lo maneja de esta forma ya que estas representaciones generalmente están llenas de ceros (no todos los términos aparecen en todos los documentos). Las matrices dispersas de `scipy` manejan de forma eficiente este tipo de datos para no consumir tanta memoria RAM.

Si quisiéramos acotar el número de características para poder hacer la conversión a un arreglo de `numpy`, podríamos utilizar algunos de los parámetros del método `fit`, así:

In [ ]:
vect = (
    CountVectorizer(max_features=1000, max_df=0.7)
    .fit(corpus_prep)
    )

Extraemos la bolsa de palabras:

In [ ]:
X = vect.transform(corpus_prep)
display(X)

Veamos cómo obtener un arreglo de `numpy` con la bolsa de características usando el método `toarray`:

In [ ]:
X_np = X.toarray()
display(X_np)

También podemos extraer el vocabulario calculado (nombres de las columnas en la matriz obtenida) usando el método `get_feature_names_out`:

In [ ]:
vocab = vect.get_feature_names_out()
display(vocab)

Podemos visualizar de mejor forma la bolsa de palabras con `pandas`:

In [ ]:
df = pd.DataFrame(columns=vocab, data=X_np)
display(df)

## **4. Term-Frequency Inverse-Document-Frequency**
---

Una de las principales desventajas de las bolsas de palabras es que asumen que todos los términos tienen igual importancia. No obstante, existen dos casos en los que algunos términos deberían tener mayor o menor importancia:

1. Palabras comunes que aparecen en todos los documentos y no aportan mucha información para distinguir un documento de otro.
2. Términos únicos y poco frecuentes que son sumamente relevantes para distinguir algunos documentos en específico.

En este ámbito, surge la necesidad de estrategias que permitan capturar este tipo de relaciones. Una de las soluciones más comunes es _term frequency - inverse document frequency (TF-IDF)_. Se trata de un método que fue propuesto como métrica para la evaluación de resultados en motores de búsqueda y se convirtió en un estándar dentro de los sistemas de recuperación de información.

<img src="https://drive.google.com/uc?export=view&id=1tOQfDdO-z2htfu0tYqmwFe1HcTdNJtY_" width="60%">

_TF-IDF_ amplía la idea de bolsas de palabras al ponderar cada término $t_j$ por un peso $w_{j}$ como se muestra a continuación:

$$
\text{TFIDF}(t_j, d_i) = \text{TF}(t_j, d_i) \cdot w_j
$$

Con esto, se puede asignar un peso menor a aquellos términos comunes entre documentos y un peso mayor a aquellos términos poco frecuentes. Una de las formas más comunes para determinar estos pesos es con la **frecuencia inversa de documento** $w_j$ que se calcula de la siguiente forma:

$$
w_j = 1 + \log{\frac{n}{1 + \text{df}(t_j)}}
$$

Donde $n$ es el número de documentos en el corpus y $\text{df}(t_j)$ es el número de documentos en los que se encuentra el término $t_j$.

Adicionalmente, una extensión de _TF-IDF_ consiste en un cambio de escala de la matriz de términos, con esto, se busca atenuar el impacto que tienen los términos que aparecen muchas veces en un documento. Esto se consigue utilizando _sub-linear scaling_ $wf(\text{TF}(t_j, d_i))$ y consiste en transformar las ocurrencias a una escala logarítmica donde los valores grandes se ven atenuados:

$$
wf(\text{TF}(t_j, d_i)) = \left\{
  \begin{array}{cl}
      1+\log{\text{TF}(t_j, d_i)} & \mathrm{si\ } \text{TF}(t_j, d_i) > 0 \\
      0 & \mathrm{si\ } \text{TF}(t_j, d_i) \le 0 \\
  \end{array}
\right.
$$

Finalmente, una versión de _TF-IDF_ con _sub-linear scaling_ se muestra a continuación:

$$
\text{TFIDF}(t_j, d_i) = wf(\text{TF}(t_j, d_i)) w_j
$$

Podemos extraer características con _TF-IDF_ usando el `TfidfVectorizer` de `sklearn`:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

El `TfidfVectorizer` funciona de una forma muy similar al `CountVectorizer`, sin embargo, incorpora algunos parámetros adicionales:

- `norm`: normalización de cada vector de características, puede usar las [normas](https://en.wikipedia.org/wiki/Minkowski_distance) `"l1"` o `"l2"`
- `use_idf`: específica si se usa ponderación _IDF_ ($w_j$).
- `smooth_idf`: suaviza el cálculo de los pesos $w_j$ al agregar uno a las frecuencias de documentos.
- `sublinear_tf`: especifica si se aplica el reescalamiento logarítmico.

Veamos cómo calcular una representación _TF-IDF_, comenzamos definiendo el vectorizador:

In [ ]:
vect = TfidfVectorizer(
    max_features=1000, max_df=0.7, norm="l2",
    sublinear_tf=True
    ).fit(corpus_prep)

Podemos extraer la representación:

In [ ]:
X = vect.transform(corpus_prep)
display(X)

Como se puede ver, el resultado también es una matriz dispersa, podemos convertirla a `numpy`:

In [ ]:
X_np = X.toarray()
display(X_np)

En este caso, los valores no corresponden directamente a conteos (números enteros), sino que obtenemos valores continuos ponderados.

## **5. Recuperación de Información**
---

Una aplicación muy típica de _TF-IDF_ consiste en la implementación de sistemas de recuperación de información. De forma general un sistema de este tipo busca satisfacer una necesidad de información (consulta o **query**) que pueda tener un usuario, para ello encuentra registros dentro de una base de datos que sean lo más parecidos a la consulta, tal y como se muestra en la siguiente figura:

<img src="https://drive.google.com/uc?export=view&id=1_RpmT0rqjksu_duWzLnaC52whQY3npwH" width="100%">

Una de las formas más simples y clásicas de implementar este tipo de sistemas es con una representación _TF-IDF_ para obtener una representación numérica y vectorial de los datos y, posteriormente, usar una **medida de similitud** para determinar qué vectores son similares entre sí. Veamos algunos ejemplos de medidas de similitud:

### **5.1. Distancia Euclidiana**
---

Una de las formas más simples para comparar la representación de dos documentos es por medio de la distancia entre los vectores o la distancia Euclidiana.

<img src="https://drive.google.com/uc?export=view&id=1NdjcZMdTDwgFTgiDwmKnPXePWgsAtvN_" width="60%">

No obstante, esta métrica no es muy apropiada con representaciones basadas en conteos _TF-IDF_, esto se debe principalmente a que en esta representación importan más los términos comunes que los términos contiguos (que en una representación de bolsa de palabras, no tienen ningún orden) y las magnitudes. Veamos un ejemplo de lo anterior, para ello, definimos tres oraciones:

In [ ]:
corpus = [
    "nuevo en america",
    "soy nuevo aqui pero america es un gran lugar para vivir",
    "la espectrometria es algo nuevo e indispensable"
]

Representamos los documentos:

In [ ]:
vect = TfidfVectorizer(
        norm=None
        )
X = vect.fit_transform(corpus).toarray()
df = pd.DataFrame(
        np.round(X, 2),
        columns=vect.get_feature_names_out()
        )
display(df)

Ahora, podemos calcular las distancias entre el primer documento y los otros dos, para ver cuál es más similar (entre menor distancia, más parecido):

In [ ]:
d1 = np.linalg.norm(X[0] - X[1])
d2 = np.linalg.norm(X[0] - X[2])
display("La distancia euclideana entre el documento 0 y el documento 1 es: {}".format(d1))
display("La distancia euclideana entre el documento 0 y el documento 2 es: {}".format(d2))

Como se puede ver, pareciera que el documento 2 es más similar al documento 0 que el documento 1 (lo cual sabemos que es erróneo).

### **5.2. Similitud Coseno**
---

Una alternativa es la **[similitud coseno](https://es.wikipedia.org/wiki/Similitud_coseno)**, la cual es más apropiada para representaciones basadas en histogramas como TF-IDF, ya que, es una medida del alineamiento de dos vectores.

<img src="https://drive.google.com/uc?export=view&id=1cipIAJzEMrs0YH2U3gz2sfqQLS3XTMjj" width="60%">

$$
\text{cos}(\mathbf{a}, \mathbf{b}) = \frac{\mathbf{a} \cdot \mathbf{b}}{|| \mathbf{a} ||~||\mathbf{b}||}
$$

Veamos el mismo ejemplo de las tres oraciones pero con la similitud coseno, primero la importamos:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

Veamos la similitud entre el primer documento y los otros dos:

In [ ]:
sim = cosine_similarity(X)
display(sim)

Veamos las similitudes:

In [ ]:
display("La distancia coseno entre el documento 0 y el documento 1 es: {}".format(sim[0, 1]))
display("La distancia coseno entre el documento 0 y el documento 2 es: {}".format(sim[0, 2]))

En este caso, los documentos son más similares entre más alto sea el resultado de la similitud coseno. Por lo tanto, podemos observar que esta medida tiene más sentido que la distancia Euclidiana para comparar representaciones _TF-IDF_.

### **5.3. BM25**
---

_Okapi BM25_ es una técnica de scoring probabilístico para texto que extiende representaciones de tipo _TF-IDF_ para recuperación de información, en especial, la fórmula de ponderación se modifica de acuerdo a tres componentes:

- Frecuencia inversa de documento.
- Curva de saturación para limitar la frecuencia de términos.
- Ponderación por longitud de cada documento.

<img src="https://drive.google.com/uc?export=view&id=1EHW4HDFjI1rZxVTTQjw8spH8Cw9Up3FU" width="100%">

Desde _Python_, podemos usar la librería `rank-bm25` para el enfoque de recuperación de información. Veamos cómo instalarla:

In [ ]:
!pip install rank-bm25

Importamos la clase que nos permitirá usar _BM25_:

In [ ]:
from rank_bm25 import BM25Okapi

Para este problema, usaremos el texto en español del conjunto de datos que teníamos cargado:

In [ ]:
corpus_spa = (
        data
        .query("language == 'Spanish'")
        .text.apply(preprocess)
        .to_list()
        )
display(corpus_spa)

Para usar _BM25_ debemos obtener una lista con los tokens por cada documento:

In [ ]:
tokens = list(map(lambda doc: doc.split(), corpus_spa))
display(tokens)

Creamos el rankeador:

In [ ]:
rank = BM25Okapi(tokens)
display(rank)

Podemos encontrar el score entre un texto dado y cada documento del corpus:

In [ ]:
query = "la comida estaba muy buena".split()
display(query)

Veamos los scores:

In [ ]:
scores = rank.get_scores(query)
display(scores)

Como podemos ver, nos da un score contra cada uno de los documentos del corpus:

In [ ]:
display(len(corpus_spa))

Esto corresponde con la longitud de los scores:

In [ ]:
display(scores.size)

Veamos el documento más similar a la consulta:

In [ ]:
idx = np.argmax(scores)
display(idx)

Veamos el documento:

In [ ]:
display(corpus_spa[idx])

Como podemos ver, el resultado es **semánticamente** cercano a la consulta.

También podemos extraer los $k$ documentos más similares a la consulta con el método `get_top_n`:

In [ ]:
top_k = rank.get_top_n(query, corpus_spa, n=10)
display(top_k)

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Working with Text Data - sklearn](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)
- [Okapi BM25](https://en.wikipedia.org/wiki/Okapi_BM25)
- _Fuente de los íconos_
    - Flaticon. Cloud free icon [PNG]. https://www.flaticon.com/free-icon/cloud_8702580
    - Flaticon. User free icon [PNG]. https://www.flaticon.com/free-icon/user_1144709
    - Flaticon. Query free icon [PNG]. https://www.flaticon.com/free-icon/query_7722230
    - Flaticon. Documents File free icon [PNG]. https://www.flaticon.com/free-icon/documents_2954086
    - Flaticon. Document free icon [PNG]. https://www.flaticon.com/free-icon/document_8965322

## Créditos
---

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*